In [1]:
# def main(args):

#     with tf.Session() as sess:

#         env = gym.make(args['env'])
#         np.random.seed(int(args['random_seed']))
#         tf.set_random_seed(int(args['random_seed']))
#         env.seed(int(args['random_seed']))

#         state_dim = env.observation_space.shape[0]
#         action_dim = env.action_space.shape[0]
#         action_bound = env.action_space.high
#         # Ensure action bound is symmetric
#         assert (env.action_space.high == -env.action_space.low)

#         actor = ActorNetwork(sess, state_dim, action_dim, action_bound,
#                              float(args['actor_lr']), float(args['tau']),
#                              int(args['minibatch_size']))

#         critic = CriticNetwork(sess, state_dim, action_dim,
#                                float(args['critic_lr']), float(args['tau']),
#                                float(args['gamma']),
#                                actor.get_num_trainable_vars())
        
#         actor_noise = OrnsteinUhlenbeckActionNoise(mu=np.zeros(action_dim))

#         if args['use_gym_monitor']:
#             if not args['render_env']:
#                 env = wrappers.Monitor(
#                     env, args['monitor_dir'], video_callable=False, force=True)
#             else:
#                 env = wrappers.Monitor(env, args['monitor_dir'], force=True)

#         train(sess, env, args, actor, critic, actor_noise)

#         if args['use_gym_monitor']:
#             env.monitor.close()

In [2]:
# import argparse
# import sys

In [3]:
# # if __name__ == '__main__':
# parser = argparse.ArgumentParser(description='provide arguments for DDPG agent')

# # agent parameters
# parser.add_argument('--actor-lr', help='actor network learning rate', default=0.0001)
# parser.add_argument('--critic-lr', help='critic network learning rate', default=0.001)
# parser.add_argument('--gamma', help='discount factor for critic updates', default=0.99)
# parser.add_argument('--tau', help='soft target update parameter', default=0.001)
# parser.add_argument('--buffer-size', help='max size of the replay buffer', default=1000000)
# parser.add_argument('--minibatch-size', help='size of minibatch for minibatch-SGD', default=64)

# # run parameters
# parser.add_argument('--env', help='choose the gym env- tested on {Pendulum-v0}', default='Pendulum-v0')
# parser.add_argument('--random-seed', help='random seed for repeatability', default=1234)
# parser.add_argument('--max-episodes', help='max num of episodes to do while training', default=50000)
# parser.add_argument('--max-episode-len', help='max length of 1 episode', default=1000)
# parser.add_argument('--render-env', help='render the gym env', action='store_true')
# parser.add_argument('--use-gym-monitor', help='record gym results', action='store_true')
# parser.add_argument('--monitor-dir', help='directory for storing gym results', default='./results/gym_ddpg')
# parser.add_argument('--summary-dir', help='directory for storing tensorboard info', default='./results/tf_ddpg')

# parser.set_defaults(render_env=False)
# parser.set_defaults(use_gym_monitor=True)

# # args = vars(parser.parse_args())

# # pp.pprint(args)

# # main(args)

In [4]:
import tensorflow as tf

In [5]:
import gym

In [6]:
from ddpg_agent import ActorNetwork, CriticNetwork, OrnsteinUhlenbeckActionNoise
import ddpg_memory

In [7]:
import numpy as np

In [8]:
# # # ===========================
# # #   Tensorflow Summary Ops
# # # ===========================

# def build_summaries():
#     episode_reward = tf.Variable(0.)
#     tf.summary.scalar("Reward", episode_reward)
#     episode_ave_max_q = tf.Variable(0.)
#     tf.summary.scalar("Qmax Value", episode_ave_max_q)

#     summary_vars = [episode_reward, episode_ave_max_q]
#     summary_ops = tf.summary.merge_all()

#     return summary_ops, summary_vars

# # ===========================
# #   Agent Training
# # ===========================

# def train(sess, env, args, actor, critic, actor_noise):

# Set up summary Ops
# summary_ops, summary_vars = build_summaries()

#     sess.run(tf.global_variables_initializer())
#     writer = tf.summary.FileWriter(args['summary_dir'], sess.graph)

#     # Initialize target network weights
#     actor.update_target_network()
#     critic.update_target_network()

#     # Initialize replay memory
#     replay_buffer = ReplayBuffer(int(args['buffer_size']), int(args['random_seed']))

#     # Needed to enable BatchNorm. 
#     # This hurts the performance on Pendulum but could be useful
#     # in other environments.
#     # tflearn.is_training(True)

#     for i in range(int(args['max_episodes'])):

#         s = env.reset()

#         ep_reward = 0
#         ep_ave_max_q = 0

#         for j in range(int(args['max_episode_len'])):

#             if args['render_env']:
#                 env.render()

#             # Added exploration noise
#             #a = actor.predict(np.reshape(s, (1, 3))) + (1. / (1. + i))
#             a = actor.predict(np.reshape(s, (1, actor.s_dim))) + actor_noise()

#             s2, r, terminal, info = env.step(a[0])

#             replay_buffer.add(np.reshape(s, (actor.s_dim,)), np.reshape(a, (actor.a_dim,)), r,
#                               terminal, np.reshape(s2, (actor.s_dim,)))

#             # Keep adding experience to the memory until
#             # there are at least minibatch size samples
#             #if replay_buffer.size() > int(args['minibatch_size']):
#             if replay_buffer.size() > int(64):
#                 s_batch, a_batch, r_batch, t_batch, s2_batch = \
#                     #replay_buffer.sample_batch(int(args['minibatch_size']))
#                     replay_buffer.sample_batch(64))

#                 # Calculate targets
#                 target_q = critic.predict_target(
#                     s2_batch, actor.predict_target(s2_batch))

#                 y_i = []
#                 for k in range(int(args['minibatch_size'])):
#                     if t_batch[k]:
#                         y_i.append(r_batch[k])
#                     else:
#                         y_i.append(r_batch[k] + critic.gamma * target_q[k])

#                 # Update the critic given the targets
#                 predicted_q_value, _ = critic.train(
#                     s_batch, a_batch, np.reshape(y_i, (int(args['minibatch_size']), 1)))

#                 ep_ave_max_q += np.amax(predicted_q_value)

#                 # Update the actor policy using the sampled gradient
#                 a_outs = actor.predict(s_batch)
#                 grads = critic.action_gradients(s_batch, a_outs)
#                 actor.train(s_batch, grads[0])

#                 # Update target networks
#                 actor.update_target_network()
#                 critic.update_target_network()

#             s = s2
#             ep_reward += r

#             if terminal:

#                 summary_str = sess.run(summary_ops, feed_dict={
#                     summary_vars[0]: ep_reward,
#                     summary_vars[1]: ep_ave_max_q / float(j)
#                 })

#                 writer.add_summary(summary_str, i)
#                 writer.flush()

#                 print('| Reward: {:d} | Episode: {:d} | Qmax: {:.4f}'.format(int(ep_reward), \
#                         i, (ep_ave_max_q / float(j))))
#                 break

In [9]:
# with tf.Session() as sess:

env = gym.make('Pendulum-v0')
np.random.seed(int(1234))
tf.set_random_seed(int(1234))
env.seed(int(123))

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
action_bound = env.action_space.high
# Ensure action bound is symmetric
assert (env.action_space.high == -env.action_space.low)

actor_noise = OrnsteinUhlenbeckActionNoise(mu=np.zeros(action_dim))

# #     if True:
# #         if not args['render_env']:
# #             env = wrappers.Monitor(
# #                 env, args['monitor_dir'], video_callable=False, force=True)
# #         else:
# #             env = wrappers.Monitor(env, args['monitor_dir'], force=True)

#     train(sess, env, args, actor, critic, actor_noise)

#     if args['use_gym_monitor']:
#         env.monitor.close()

/home/arasdar/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [10]:
buffer_size = 64
random_seed = 1234
gamma = 0.99
tau = 1e-3
actor_lr = 1e-4
critic_lr = 1e-3
max_episodes = 1000
max_episode_len = 1000
batch_size = 64

In [11]:
graph = tf.reset_default_graph()

with tf.Session(graph=graph) as sess:
    
    sess.run(tf.global_variables_initializer())

    # Initialize target network weights
    actor = ActorNetwork(sess, state_dim, action_dim, action_bound,
                         actor_lr, tau, batch_size)

    critic = CriticNetwork(sess, state_dim, action_dim, 
                           critic_lr, tau, gamma,
                           actor.get_num_trainable_vars())
    actor.update_target_network()
    critic.update_target_network()

#     # Initialize replay memory
#     replay_buffer = ReplayBuffer(buffer_size, random_seed)

#     # Needed to enable BatchNorm. 
#     # This hurts the performance on Pendulum but could be useful
#     # in other environments.
#     # tflearn.is_training(True)

#     for i in range(max_episodes):

#         s = env.reset()

#         ep_reward = 0
#         ep_ave_max_q = 0

#         for j in range(max_episode_len):

#             #if args['render_env']:
#             #env.render()

#             # Added exploration noise
#             #a = actor.predict(np.reshape(s, (1, 3))) + (1. / (1. + i))
#             a = actor.predict(np.reshape(s, (1, actor.s_dim))) + actor_noise()

#             s2, r, terminal, info = env.step(a[0])

#             replay_buffer.add(np.reshape(s, (actor.s_dim,)), np.reshape(a, (actor.a_dim,)), r,
#                               terminal, np.reshape(s2, (actor.s_dim,)))

#             # Keep adding experience to the memory until
#             # there are at least minibatch size samples
#             #if replay_buffer.size() > int(args['minibatch_size']):
#             if replay_buffer.size() > batch_size:
#                 s_batch, a_batch, r_batch, t_batch, s2_batch = replay_buffer.sample_batch(batch_size)
#                     #replay_buffer.sample_batch(int(args['minibatch_size']))

#                 # Calculate targets
#                 target_q = critic.predict_target(
#                     s2_batch, actor.predict_target(s2_batch))

#                 y_i = []
#                 for k in range(batch_size):
#                     if t_batch[k]:
#                         y_i.append(r_batch[k])
#                     else:
#                         y_i.append(r_batch[k] + critic.gamma * target_q[k])

#                 # Update the critic given the targets
#                 predicted_q_value, _ = critic.train(
#                     s_batch, a_batch, np.reshape(y_i, batch_size, 1))

#                 ep_ave_max_q += np.amax(predicted_q_value)

#                 # Update the actor policy using the sampled gradient
#                 a_outs = actor.predict(s_batch)
#                 grads = critic.action_gradients(s_batch, a_outs)
#                 actor.train(s_batch, grads[0])

#                 # Update target networks
#                 actor.update_target_network()
#                 critic.update_target_network()

#             s = s2
#             ep_reward += r

#             if terminal:

# #                 summary_str = sess.run(summary_ops, feed_dict={
# #                     summary_vars[0]: ep_reward,
# #                     summary_vars[1]: ep_ave_max_q / float(j)
# #                 })

# #                 writer.add_summary(summary_str, i)
# #                 writer.flush()

#                 print('| Reward: {:d} | Episode: {:d} | Qmax: {:.4f}'.format(int(ep_reward), \
#                         i, (ep_ave_max_q / float(j))))
#                 break

FailedPreconditionError: Attempting to use uninitialized value FullyConnected_3/W
	 [[Node: FullyConnected_3/W/read = Identity[T=DT_FLOAT, _class=["loc:@FullyConnected_3/W"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](FullyConnected_3/W)]]

Caused by op 'FullyConnected_3/W/read', defined at:
  File "/home/arasdar/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/arasdar/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/arasdar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/arasdar/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/arasdar/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/arasdar/anaconda3/lib/python3.6/site-packages/tornado/